In [1]:
# 필요한 패키지 불러오기
import pandas as pd
from konlpy.tag import Okt 
okt = Okt()
import gensim
import gensim.corpora as corpora
import os
# os.environ["PYTHONIOENCODING"] = "utf-8”
import pyLDAvis.gensim

In [2]:
# 저장한 데이터 불러오기
df = pd.read_excel('(2)apple_all_airpods.xlsx')
df.head(20)

,ratings,reviews,label
0,5,비싸서 고민되면 사는게 맞다. 1. 배송속도 / 2. 제조년월 / 3. 포장 품질 ...,1
1,4,배송은 엉망이지만 사용 후기는 아주 만족 와.. 대박 방금 받았는데...엄청 안전하...,1
2,5,한번도 안 써본 사람은 있어도 한번만 쓴 사람은 없다. 아이폰6s에서 8로 바꿨더니...,1
3,4,진심 이건 신세계다!!!!!!!!@_@ 그동안 쿠팡에서 로켓배송을 시켜도 한번도 박...,1
4,5,애플의 감성을 담은 블루투스 이어폰 무선 블루투스 이어폰의 새강자 '에어팟'그렇다....,1
5,5,그냥 이어폰이 아닌 생활필수품 사용해보고 후기 남기고 싶어서 좀 지나고 나서야 후기...,1
6,5,가격도 나름 온라인에서 착하고 배송이 짱 빨라서 추천 마침 봄맞이 로켓페이 행사도 ...,1
7,5,역시 애플!!! 솔직히 보기에 선없는 번들 이어폰 끼고 다니는듯해서 망설이다 사용하...,1
8,4,엄청 비쌈; 연결도 잘되고 좋아요!로켓배송이라 여기서 돈 더쥬고 샀는데 괜찮아요정품...,1
9,5,외 않사요? 사세요. 왜 다들 그 큰 돈을 들여가며 이 하얀 콩나물을 샀는지 알 것...,1


In [3]:
positiveDf = df[df['label'] == 1]
positiveDf

,ratings,reviews,label
0,5,비싸서 고민되면 사는게 맞다. 1. 배송속도 / 2. 제조년월 / 3. 포장 품질 ...,1
1,4,배송은 엉망이지만 사용 후기는 아주 만족 와.. 대박 방금 받았는데...엄청 안전하...,1
2,5,한번도 안 써본 사람은 있어도 한번만 쓴 사람은 없다. 아이폰6s에서 8로 바꿨더니...,1
3,4,진심 이건 신세계다!!!!!!!!@_@ 그동안 쿠팡에서 로켓배송을 시켜도 한번도 박...,1
4,5,애플의 감성을 담은 블루투스 이어폰 무선 블루투스 이어폰의 새강자 '에어팟'그렇다....,1
...,...,...,...
7255,5,등록된 헤드라인이 없습니다 등록된 리뷰내용이 없습니다,1
7256,5,등록된 헤드라인이 없습니다 등록된 리뷰내용이 없습니다,1
7257,5,등록된 헤드라인이 없습니다 등록된 리뷰내용이 없습니다,1
7258,5,등록된 헤드라인이 없습니다 등록된 리뷰내용이 없습니다,1


In [4]:
# 리뷰 데이터만 가져오기
review = positiveDf['reviews']

# 빈 리스트를 만들어 명사 토큰을 추출해 저장한다.
review_noun_tk = []
for n in review:
    review_noun_tk.append(okt.nouns(n)) # 명사 형태소만 출력

In [5]:
#한글자로만 추출된 데이터를 제거하기
review_noun_tk2 = []
for d in review_noun_tk:
    item = [i for i in d if len(i) > 1] # 뽑아낸 토큰 중 크기가 1 이상만 추출
    review_noun_tk2.append(item)

## LDA 모델 구축

In [14]:
# 명사 데이터를 사전화 하여 빈도 필터링을 진행한다.
dict = corpora.Dictionary(review_noun_tk2)

In [15]:
# '등록 헤드라인 등록 리뷰 용이'는 리뷰가 없으면 자동생성되는 내용임 
# '버즈'에 대한 크롤링 내용이기에 압도적으로 많을 수 뿐이 없기에 이러한 필요없는 명사 삭제처리
removeWords = ['등록', '헤드라인', '리뷰', '용이', '제품', '에어팟']

ids_to_remove = [dict.token2id[word] for word in removeWords if word in dict.token2id]
dict.filter_tokens(bad_ids=ids_to_remove)

In [16]:
# 딕셔너리화 성공~
print(dict.token2id)

{'가격': 0, '가끔': 1, '가장': 2, '각도': 3, '개봉': 4, '개인': 5, '거리': 6, '거주지': 7, '걱정': 8, '것임': 9, '결과': 10, '고민': 11, '과정': 12, '구매': 13, '궁금증': 14, '귓볼': 15, '기간': 16, '난리': 17, '날짜': 18, '내외': 19, '넘버': 20, '년월': 21, '다른': 22, '다시': 23, '대각선': 24, '대해': 25, '딜레이': 26, '때문': 27, '뚜껑': 28, '라운드': 29, '로그인': 30, '마이크': 31, '매우': 32, '먼지': 33, '모름': 34, '모바일': 35, '무작위': 36, '문건': 37, '문제': 38, '밀착': 39, '바람': 40, '반응': 41, '발생': 42, '방식': 43, '배송': 44, '배터리': 45, '배틀': 46, '보고': 47, '보시': 48, '보증': 49, '부분': 50, '부산': 51, '부터': 52, '불규칙': 53, '블루투스': 54, '비닐': 55, '사각형': 56, '사람': 57, '사용': 58, '상품': 59, '생각': 60, '소리': 61, '소음': 62, '속도': 63, '수준': 64, '시간': 65, '시리얼': 66, '신호': 67, '실행': 68, '아이디': 69, '아이폰': 70, '애플': 71, '약간': 72, '어보': 73, '업체': 74, '연결': 75, '완료': 76, '유입': 77, '은근': 78, '음질': 79, '이상은': 80, '이상인': 81, '이어폰': 82, '입자': 83, '자석': 84, '재생': 85, '적임': 86, '정도': 87, '정말': 88, '제일': 89, '제조': 90, '조금': 91, '주문': 92, '준비': 93, '증상': 94, '지연': 95, '지인': 96, '착용': 97, '초음파': 98

In [17]:
# 단어의 빈도수를 확인하기 위해 doc2bow 로 변환한다.
# BoW 모델은 문서의 단어의 빈도수를 표현해준다.
corpus = [dict.doc2bow(word) for word in review_noun_tk2]

In [18]:
# corpus 출력을 통해 (word_id, work_count)의 BoW 구성을 확인
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 4), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 3), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 2), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 3), (55, 1), (56, 1), (57, 2), (58, 2), (59, 3), (60, 1), (61, 2), (62, 2), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 4), (80, 1), (81, 1), (82, 3), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 3), (91, 4), (92, 2), (93, 1), (94, 1), (95, 1), (96, 1), (97, 2), (98, 1), (99, 1), (100, 2), (101, 1), (102, 2), (103, 1), (104, 2), (105, 2), (106, 1), (107, 1), (108, 1), (109, 3), (110, 1)

In [19]:
# 토픽의 갯수 k 개 생성
k = 3

#gensim 라이브러리를 사용해 lda 모델을 생성하는 코드 각 옵션은 만들어놓은 
#corpus lda모델
#iterations 각 문서의 주제 할당을 반복하는 횟수
#num_topics 는 위에 지정한 k 값으로 토픽의 갯수
#id2word 각 토큰의 id에 해당하는 토큰을 연결해준다
#passes는 앞서 넣은 corpus를 학습 반복하는 횟수이다 수가 높아지면 더 잘 학습하지만 그만큼 시간이 증가한다.
#workers 쓰레드의 수 (작업자의 수)이다. 보통은 본인의 컴퓨터에 맞춰 설정하게 된다.
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, iterations = 12, num_topics = k, id2word = dict, passes = 5, workers = 10)

In [20]:
print(lda_model.print_topics(num_topics = k, num_words = 15))

[(0, '0.029*"구매" + 0.027*"프로" + 0.025*"음질" + 0.021*"배송" + 0.015*"세대" + 0.013*"사용" + 0.012*"가격" + 0.012*"애플" + 0.012*"이어폰" + 0.011*"노이즈" + 0.011*"기능" + 0.011*"슬링" + 0.010*"역시" + 0.010*"고민" + 0.009*"아주"'), (1, '0.038*"사용" + 0.031*"세대" + 0.025*"이어폰" + 0.019*"구매" + 0.016*"프로" + 0.014*"음질" + 0.011*"고민" + 0.011*"착용" + 0.010*"가격" + 0.010*"아이폰" + 0.010*"선물" + 0.009*"정말" + 0.008*"애플" + 0.006*"배터리" + 0.006*"배송"'), (2, '0.024*"세대" + 0.015*"충전" + 0.014*"정품" + 0.012*"구매" + 0.011*"이어폰" + 0.011*"사용" + 0.010*"배송" + 0.010*"쿠팡" + 0.010*"가격" + 0.010*"음질" + 0.010*"소리" + 0.010*"애플" + 0.008*"고민" + 0.008*"그냥" + 0.008*"생각"')]


In [21]:
# lda모델을 디스플레이로 출력
lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict)
pyLDAvis.display(lda_vis)

### 에어팟 긍정 토픽분석을 통해 얻을 수 있는 정보
1. 긍정리뷰에 에서도 비싼 가격에 대한 이야기가 많이 나옴을 확인가능
2. 긍정토픽에는 애플의 핵심기능 중 하나인 노이즈캔슬링에 대한 이야기가 많음
3. 주로 가격이 크게 비싸지 않기에 다른 사람에게 선물을 한다는 리뷰가 많음